# 2019 MolSSI Summer School QM project: semiempirical model of Argon

## 1. Introduction

In this project, we will simulate a cluster of Argon atoms using quantum mechanics (QM) to calculate their total energy. First-principles QM simulations are complicated and expensive, and a quick implementation would rely on a substantial amount of pre-existing software infrastructure. Instead, we will implement a much simpler semiempirical QM simulation that has been designed and parameterized to reproduce first-principles QM data using a minimal model. We can then limit our external dependencies to the standard mathematical functionality of Python: NumPy to build and solve matrix and tensor equations and SymPy to compute analytical derivatives as needed:

In [1]:
import numpy as np
import sympy

As is typically the case in quantum chemistry, we will treat the atomi nuclei as classical point charges,

$$ \vec{r}_i = (x_i , y_i, z_i). $$

While we would normally read these from a file or pass them in as an argument to a function, we will just specify a set of atomic coordinates in this example:

In [2]:
atomic_coordinates = np.array([ [0.0,0.0,0.0] ])
#atomic_coordinates = np.array([ [0.0,0.0,0.0], [3.0,4.0,5.0] ])
number_of_atoms = len(atomic_coordinates)

np.set_printoptions(precision=1)
print(number_of_atoms)
print(atomic_coordinates)

1
[[0. 0. 0.]]


Similarly, we will only calculate and print the total energy to the screen, whereas more useful software would probably print output data to a file or return it from a function call. You will add such features as you refactor this software. All physical quantities in this project will be specified in Hartree atomic units, where the bohr is the unit of length and the hartree is the unit of energy.

## 2. Model Hamiltonian

This semiempirical model will combine some standard concepts and methods used in physics and chemistry. First, it will use a minimal number of electronic degrees of freedom. Because Argon is a noble gas, it interacts primarily through London dispersion forces that are mediated by quantum dipole fluctuations. The lowest energy dipole transition is from the occupied $3p$ states to the unoccupied $4s$ state, and we will include these 4 atomic orbitals per atom. Similarly, we will use a multipole expansion to simplify electronic excitations and retain only the monopole and dipole terms, which also restricts electronic polarization to 4 degrees of freedom per atom. We will use $\{s, p_x, p_y, p_z\}$ to label both sets of basis functions on each atom.

In [3]:
orbital_types = ['s', 'px' ,'py', 'pz']
orbitals_per_atom = len(orbital_types)

p_orbitals = orbital_types[1:]
print(p_orbitals)

['px', 'py', 'pz']


Within our minimal basis, we will describe the electronic state as the ground state of a quantum many-body Hamiltonian $\hat{H}$. As is standard in quantum chemistry, we begin by writing the Hamiltonian in second quantization notation,

$$ \hat{H} = E_{\mathrm{ion}} + \sum_{p,q} \sum_{\sigma \in \{ \uparrow , \downarrow \} } h_{p,q} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{q,\sigma} + \tfrac{1}{2}\sum_{p,q,r,s} \sum_{\sigma,\sigma' \in \{ \uparrow , \downarrow \} } V_{p,q,r,s} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{r,\sigma'}^{\dagger} \hat{a}_{s,\sigma'} \hat{a}_{q,\sigma} , $$

where $\hat{a}_{p,\sigma}^{\dagger}$ and $\hat{a}_{p,\sigma}$ are the electron raising and lowering operators for an atomic orbital index $p$ and spin $\sigma$. We will not be using $\hat{H}$ itself in our calculations, but we will make use of the coefficient tensors $h_{p,q}$ and $V_{p,q,r,s}$. In first-principles calculations, each element of $h_{p,q}$ and $V_{p,q,r,s}$ would require the evaluation of a complicated integral. In our semiempirical model, we will set most of them to zero and assign a simple analytical form to the rest of them. We will also assume that the atomic orbitals are all orthogonal, which avoids having to deal with overlap matrices. The notation being used here is mostly consistent with modern quantum chemistry notation, but some objects, particularly $V_{p,q,r,s}$, have multiple conventions in practice.

The atomic orbital index contains information about which atom the orbital is centered on and the type of orbital it is. We will often extract these individual pieces of information using $\mathrm{atom}(p)$ to denote the atom and $\mathrm{orb}(p)$ to denote the orbital type. This is the first of many instances in this project where we could either represent something as a pre-tabulate list or a function. We will always make the simpler choice, in this case functions:

In [4]:
def atom(ao_index):
    '''Returns the atom index part of an atomic orbital index.'''
    return ao_index // orbitals_per_atom

def orb(ao_index):
    '''Returns the orbital type of an atomic orbital index.'''
    orb_index = ao_index % orbitals_per_atom
    return orbital_types[orb_index]

def ao_index(atom_p,orb_p):
    '''Returns the atomic orbital index for a given atom index and orbital type.'''
    p = atom_p*orbitals_per_atom
    p += orbital_types.index(orb_p)
    return p

print("1st atom index =", atom(0))
print("1st orbital type =", orb(0))
print("ao index of s orbital on 1st atom =", ao_index(0, 's'))

1st atom index = 0
1st orbital type = s
ao index of s orbital on 1st atom = 0


### A. Slater-Koster tight-binding model

We will describe the kinetic energy of electrons using a simplified [Slater-Koster tight-binding method](https://en.wikipedia.org/wiki/Tight_binding). Because of the symmetry of atomic orbitals and the translational invariance of the kinetic energy operator, there are 4 distinct, distance-dependent "hopping" energies that characterize the interatomic kinetic energy between s and p orbitals:

![s-p hopping diagram](hopping_cases.png)

All other atomic orientations can be related to these cases by a change of coordinates. While it is compatible with very general functional forms, we will use a Gaussian form to simplify the model and its implementation. The parameters of this simple version are a common length scale $r_{\mathrm{SK}}$ and the strength of each hopping energy:

In [5]:
r_sk = 6.0
t_sss = -0.002
t_sps = -0.001
t_pps = -0.001
t_ppp = -0.001

The distance-dependent "hopping" matrix elements are then defined by

$$ t_{o,o'}^{\mathrm{SK}}(\vec{r}) = \exp(1-r^2/r_{\mathrm{SK}}^2) \times \begin{cases}
 t_{sss} , & o = o' = s \\
 [\vec{o}' \cdot (\vec{r}/r_{\mathrm{SK}})] t_{sps}, & o = s \ \& \ o' \in \{p_x, p_y, p_z\} \\
 -[\vec{o} \cdot (\vec{r}/r_{\mathrm{SK}})] t_{sps} , & o' = s \ \& \ o \in \{p_x, p_y, p_z\} \\
 (r^2/r_{\mathrm{SK}}^2)\,(\vec{o} \cdot \vec{o}')  t_{ppp} - [\vec{o} \cdot (\vec{r}/r_{\mathrm{SK}})] [\vec{o}' \cdot (\vec{r}/r_{\mathrm{SK}})] (t_{pps} + t_{ppp}), & o,o' \in \{p_x, p_y, p_z\}
 \end{cases} $$
 
where $o$ and $o'$ are the orbital types of the 1st and 2nd atoms and $\vec{r}$ is a vector pointing from the 2nd atom to the 1st atom. We are assigning direction vectors to the p orbitals, $\vec{p}_x \equiv (1,0,0)$, $\vec{p}_y \equiv (0,1,0)$, and $\vec{p}_z \equiv (0,0,1)$, to simplify the notation. This project has multiple case-based formulas, and we will implement them using a code structure similar to each formula:

In [6]:
vec = { 'px':[1,0,0], 'py':[0,1,0], 'pz':[0,0,1] }

def hopping_matrix_element(o1,o2,r12):
    '''Returns the hopping matrix element for a pair of orbitals of types o1 & o2 separated by a vector r12.'''
    r12_rescaled = r12 / r_sk
    r12_squared = np.dot(r12_rescaled, r12_rescaled)
    ans = np.exp( 1.0 - r12_squared )
    if o1 == 's' and o2 == 's':
        ans *= t_sss
    if o1 == 's' and o2 in p_orbitals:
        ans *= np.dot(vec[o2], r12_rescaled) * t_sps
    if o2 == 's' and o1 in p_orbitals:
        ans *= -np.dot(vec[o1], r12_rescaled)* t_sps
    if o1 in p_orbitals and o2 in p_orbitals:
        ans *= ( r12_squared * np.dot(vec[o1], vec[o2]) * t_ppp
                 - np.dot(vec[o1], r12_rescaled) * np.dot(vec[o2], r12_rescaled) * (t_pps + t_ppp) )
    return ans

print("vec[px] =",vec['px'])
print("hopping test",hopping_matrix_element('s','px',np.array([r_sk,0.0,0.0])))

vec[px] = [1, 0, 0]
hopping test -0.001


### B. Coulomb interaction

The nuclear charge of Argon is 18, but our model combines the nucleus and the 12 neglected electrons ($1s^2$, $2s^2$, $2p^6$, and $3s^2$) as an ionic point charge with $Z = 6$. For the purpose of electrostatics, we will describe the valence electrons of each Argon atom as Gaussian charges and their derivatives, which form a multipole expansion. The width of these Gaussians, $r_0$, is the first parameter of our semiempirical model:

In [7]:
ionic_charge = 6.0
r0 = 6.5

The electron-ion and electron-electron interactions have the same functional form, but the electron-electron interaction has an wider effective width of $\sqrt{2}r_0$,

$$ V^{\mathrm{II}}(r) = \frac{Z^2}{r} \\
   V^{\mathrm{eI}}(r) = -Z \frac{\mathrm{erf}(r/r_0)}{r} \\
   V^{\mathrm{ee}}(r) = \frac{\mathrm{erf}(r/(\sqrt{2}r_0))}{r}. $$

These functions can be constructed using SymPy, which will allow us to evaluate analytical derivatives automatically:

In [8]:
x1, y1, z1, x2, y2, z2, r = sympy.symbols('x1 y1 z1 x2 y2 z2 r')
r12 = sympy.sqrt( (x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1e-16)

erf_kernel = sympy.erf(r) / r
erf_kernel_close = erf_kernel.series(r,0,3).removeO()

v_ii_kernel = ionic_charge**2 / r12
v_ei_kernel = -ionic_charge*erf_kernel.subs(r, r12/r0) / r0
v_ee_kernel = erf_kernel.subs(r, r12/(np.sqrt(2.0)*r0)) / (np.sqrt(2.0)*r0)

v_ei_kernel_close = -ionic_charge*erf_kernel_close.subs(r, r12/r0) / r0
v_ee_kernel_close = erf_kernel_close.subs(r, r12/(np.sqrt(2.0)*r0)) / (np.sqrt(2.0)*r0)

def eval_kernel(kernel):
    '''Returns a NumPy function that efficiently evaluates the SymPy function input.'''
    return sympy.lambdify(([x1,y1,z1], [x2,y2,z2]), kernel)

print(erf_kernel)
print(erf_kernel_close)

erf(r)/r
-2*r**2/(3*sqrt(pi)) + 2/sqrt(pi)


SymPy is not smart enough to take limits automatically when they are needed, so we need to do this ourselves to avoid numerical singularities. It is then straightforward (but slow) to "compile" a function from SymPy to NumPy to evaluate formulas. For example, the ion-ion energy in $\hat{H}$,

$$ E_{\mathrm{ion}} = \sum_{i < j} V^{\mathrm{II}}(|\vec{r}_i - \vec{r}_j|) $$

can be implemented as:

In [9]:
def calculate_energy_ion(atomic_coordinates):
    '''Returns the ionic contribution to the total energy for an input list of atomic coordinates.'''
    energy_ion = 0.0
    v_ii = eval_kernel( v_ii_kernel )
    for i, r_i in enumerate(atomic_coordinates):
        for j, r_j in enumerate(atomic_coordinates):
            if i < j:
                energy_ion += v_ii(r_i, r_j)
    return energy_ion

energy_ion = calculate_energy_ion(atomic_coordinates)
print(energy_ion)

0.0


For functions involving electrons, we need to define a differential operator that will generate our multipole expansions,

$$ \Delta_p = \begin{cases} 1 , & \mathrm{orb}(p) = s \\
d/d x_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_x \\
d/d y_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_y \\
d/d z_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_z
\end{cases}, $$

In [10]:
def grad1(kernel,orb1):
    '''Returns the input interaction kernel with a multipole generator in the 1st coordinate applied to it.'''
    if orb1 == 's':
        return kernel
    if orb1 == 'px':
        return sympy.diff(kernel, x1)
    if orb1 == 'py':
        return sympy.diff(kernel, y1)
    if orb1 == 'pz':
        return sympy.diff(kernel, z1)

def grad2(kernel,orb2):
    '''Returns the input interaction kernel with a multipole generator in the 2nd coordinate applied to it.'''
    if orb2 == 's':
        return kernel
    if orb2 == 'px':
        return sympy.diff(kernel, x2)
    if orb2 == 'py':
        return sympy.diff(kernel, y2)
    if orb2 == 'pz':
        return sympy.diff(kernel, z2)

print("function:\n", v_ee_kernel)
print("x1 derivative:\n", grad1(v_ee_kernel,'px'))
print("x1 & x2 derivatives:\n", grad2(grad1(v_ee_kernel,'px'),'px'))

function:
 1.0*erf(0.108785658644084*sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16))/sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16)
x1 derivative:
 1.0*(-x1 + x2)*erf(0.108785658644084*sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16))/((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16)**(3/2) + 0.217571317288168*(x1 - x2)*exp(-0.0118343195266272*(x1 - x2)**2 - 0.0118343195266272*(y1 - y2)**2 - 0.0118343195266272*(z1 - z2)**2)/(sqrt(pi)*((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16))
x1 & x2 derivatives:
 0.217571317288168*(-x1 + x2)**2*exp(-0.0118343195266272*(x1 - x2)**2 - 0.0118343195266272*(y1 - y2)**2 - 0.0118343195266272*(z1 - z2)**2)/(sqrt(pi)*((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16)**2) + 1.0*(-x1 + x2)*(3*x1 - 3*x2)*erf(0.108785658644084*sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16))/((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1.0e-16)**(5/2) + 0.217571317288168*(0.0236686390532544*x1 - 0.0236686390532544

both for the vector of electron-ion potential matrix elements,

$$ V^{\mathrm{eI}}_p = \sum_{i} \Delta_p V^{\mathrm{eI}}(|\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i|) , $$

In [11]:
def calculate_potential_vector(atomic_coordinates):
    '''Returns the electron-ion potential energy vector for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    potential_vector = np.zeros(ndof)
    for orb_p in orbital_types:
        v_ei = eval_kernel( grad1(v_ei_kernel, orb_p) )
        v_ei_close = eval_kernel( grad1(v_ei_kernel_close, orb_p) )
        for atom_p,r_p in enumerate(atomic_coordinates):
            p = ao_index(atom_p, orb_p)
            for atom_i,r_i in enumerate(atomic_coordinates):
                if atom_i != atom_p:
                    potential_vector[p] += v_ei(r_p, r_i)
                else:
                    potential_vector[p] += v_ei_close(r_p,r_i)
    return potential_vector

potential_vector = calculate_potential_vector(atomic_coordinates)
print(potential_vector)

[-1.  0.  0.  0.]


and the matrix of electron-electron interaction matrix elements,

$$ V^{\mathrm{ee}}_{p,q} = \Delta_p \Delta_q V^{\mathrm{ee}}(|\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)}|) . $$

In [12]:
def calculate_interaction_matrix(atomic_coordinates):
    '''Returns the electron-electron interaction energy matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    interaction_matrix = np.zeros( (ndof,ndof) )
    for orb_p in orbital_types:
        for orb_q in orbital_types:
            v_ee = eval_kernel( grad2(grad1(v_ee_kernel, orb_p), orb_q) )
            v_ee_close = eval_kernel( grad2(grad1(v_ee_kernel_close, orb_p), orb_q) )
            for atom_p,r_p in enumerate(atomic_coordinates):
                p = ao_index(atom_p, orb_p)
                for atom_q,r_q in enumerate(atomic_coordinates):
                    q = ao_index(atom_q, orb_q)
                    if atom_p != atom_q:
                        interaction_matrix[p,q] = v_ee(r_p, r_q)
                    else:
                        interaction_matrix[p,q] = v_ee_close(r_p, r_q)
    return interaction_matrix

interaction_matrix = calculate_interaction_matrix(atomic_coordinates)
print(interaction_matrix)

[[ 0.1 -0.  -0.  -0. ]
 [-0.   0.   0.   0. ]
 [-0.   0.   0.   0. ]
 [-0.   0.   0.   0. ]]


### C. Multipole decomposition

To define $V_{p,q,r,s}$ based on the Coulomb matrix elements $V_{p,q}^{\mathrm{ee}}$, we need to define a mapping from products of atomic orbitals to a linear combination of terms in the multipole expansion of electronic charge. Because the atomic orbitals are normalized, their monopole coefficient with themselves is 1 (i.e. they have unit charge). Because of orthogonality, there is no monopole term for either intra-atomic or inter-atomic transitions between atomic orbitals. We will ignore dipole transitions between atoms, which corresponds to the [neglect of diatomic differential overlap](https://en.wikipedia.org/wiki/NDDO) (NDDO) approximation that is commonly used in semiempirical quantum chemistry. All that remains is the intra-atomic s-p transition, and we will use a model parameter, $D$, to define its dipole strength:

In [13]:
dipole_strength = 10.0

These transformation rules between atomic orbitals and multipole moments can be summarized pictorially,

![s-p multipole diagram](multipole_cases.png)

or mathematically as a 3-index tensor $\chi_{p,q,r}$ where $p$ and $q$ are the atomic orbital indices and $r$ is the multipole moment index,

$$ \chi_{p, q, r} = \begin{cases} 1, & \mathrm{orb}(p) = \mathrm{orb}(q) \ \& \ \mathrm{orb}(r) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\ 
 D , & \mathrm{orb}(q) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(p) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 D , & \mathrm{orb}(p) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(q) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 0, & \mathrm{otherwise} \end{cases} . $$

In [14]:
def chi_on_atom(o1,o2,o3):
    '''Returns the value of the chi tensor for 3 orbital indices on the same atom.'''
    if o1 == o2 and o3 == 's':
        return 1.0
    if o1 == o3 and o3 in p_orbitals and o2 == 's':
        return dipole_strength
    if o2 == o3 and o3 in p_orbitals and o1 == 's':
        return dipole_strength
    return 0.0

The multipole expansion plays the same role as an auxiliary basis set in modern resolution-of-identity (RI) methods that are used to accelerate large quantum chemistry simulations. The primary purpose of these methods is to decompose $V_{p,q,r,s}$ into a low-rank factored form, which is

$$ V_{p,q,r,s} = \sum_{t,u} \chi_{p,q,t} V_{t,u}^{\mathrm{ee}} \chi_{r,s,u} $$

in our semiempirical model. Unlike the previous vectors and matrices that we have constructed, the $\chi$ and $V$ tensors are sparse, meaning that most of their entries are zero. Without a more clever implementation, it can be computationally slow and wasteful to store and compute with these zero values. We have instead implemented $\chi$ as a function that computes non-zero tensor elements as needed. This is analogous to "direct" quantum chemistry methods that recompute integrals on-the-fly instead of storing them. We will also manipulate formulas to use the factored form of $V_{p,q,r,s}$, so that we don't need to compute or store it explicitly either.

### D. 1-body Hamiltonian

The 1-body Hamiltonian coefficients $h_{p,q}$ combine many of the components that we have already discussed and implemented. It also contains the final two parameters of our semiempirical model, the orbital energies of the s and p atomic orbitals:

In [15]:
energy_s = 0.6
energy_p = 0.0

The diagonal matrix elements contain these energies and contributions from the electron-ion potential, while the off-diagonal matrix elements contain hopping energies defined by the Slater-Koster tight-binding model,

$$ h_{p,q} = \begin{cases}
 E_{\mathrm{orb}(p)} \delta_{\mathrm{orb}(p),\mathrm{orb}(q)} + \sum_{r} \chi_{p,q,r} V_{r}^{\mathrm{eI}} , & \mathrm{atom}(p) = \mathrm{atom}(q) \\
 t^{\mathrm{SK}}_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)})
  , & \mathrm{atom}(p) \neq \mathrm{atom}(q)
  \end{cases}. $$

In [16]:
orbital_energy = { 's':energy_s, 'px':energy_p, 'py':energy_p, 'pz':energy_p }

def calculate_hamiltonian_matrix(atomic_coordinates):
    '''Returns the 1-body Hamiltonian matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    hamiltonian_matrix = np.zeros( (ndof,ndof) )
    potential_vector = calculate_potential_vector(atomic_coordinates)
    for p in range(ndof):
        for q in range(ndof):
            if atom(p) == atom(q):
                hamiltonian_matrix[p,q] = 0.0
                if orb(p) == orb(q):
                    hamiltonian_matrix[p,q] += orbital_energy[orb(p)]
                for orb_r in orbital_types:
                    r = ao_index(atom(p),orb_r)
                    hamiltonian_matrix[p,q] += chi_on_atom(orb(p), orb(q), orb_r) * potential_vector[r]
            if atom(p) != atom(q):
                r_pq = atomic_coordinates[atom(p)] - atomic_coordinates[atom(q)]
                hamiltonian_matrix[p,q] = hopping_matrix_element(orb(p), orb(q), r_pq)
    return hamiltonian_matrix

hamiltonian_matrix = calculate_hamiltonian_matrix(atomic_coordinates)
print("hamiltonian matrix =\n",hamiltonian_matrix)

hamiltonian matrix =
 [[-0.4  0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.  -1. ]]


We have now fully specified the many-body Hamiltonian $\hat{H}$, and we can now move on to approximating and calculating some of its physical properties.

## 3. Hartree-Fock theory

Even with a simple model for $\hat{H}$, we cannot calculate its ground state energy exactly for more than a few Argon atoms. Instead, we will use the [Hartree-Fock approximation](https://en.wikipedia.org/wiki/Hartree–Fock_method), which restricts the ground-state wavefunction to a single Slater determinant. We will find the Slater determinant with the lowest total energy, but a more general wavefunction will usually have an even lower energy. In the next section, we will use many-body perturbation theory to improve our estimate of the total energy.

The central objects of Hartree-Fock theory are the 1-electron density matrix $\rho_{p,q}$ and the Fock matrix $f_{p,q}$. These two matrices depend on each other, which defines a nonlinear set of equations that we must solve iteratively. Iteratively solving these equations is usually referred to as the self-consistent field (SCF) cycle. For this to converge, we must start from a reasonable initial guess for $\rho_{p,q}$. We will initialize it to the density matrix for isolated Argon atoms,

$$ \rho_{p,q}^{\mathrm{atom}} = \begin{cases} 
   1, & p = q \ \& \ \mathrm{orb}(p) \in \{ p_x, p_y, p_z \} \\
   0, & \mathrm{otherwise}
\end{cases} $$

In [17]:
orbital_occupation = { 's':0, 'px':1, 'py':1, 'pz':1 }

def calculate_atomic_density_matrix(atomic_coordinates):
    '''Returns a trial 1-electron density matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    density_matrix = np.zeros( (ndof,ndof) )
    for p in range(ndof):
        density_matrix[p,p] = orbital_occupation[orb(p)]
    return density_matrix

density_matrix = calculate_atomic_density_matrix(atomic_coordinates)
print(density_matrix)

[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


Because of spin symmetry, we use the same $\rho_{p,q}$ for each electron spin type, which reduces the sum over spin types in $\hat{H}$ to degeneracy pre-factors. Thus, half of the electrons are spin-up and half are spin-down.

The Fock matrix is defined by the density matrix,

$$ \begin{align} f_{p,q} & = h_{p,q} + \sum_{r,s} ( 2 V_{p,q,r,s} - V_{r,q,p,s} )\rho_{r,s} \\
                         & = h_{p,q} + \sum_{r,s,t,u} ( 2 \chi_{p,q,t} \chi_{r,s,u} - \chi_{r,q,t} \chi_{p,s,u} )
                             V_{t,u}^{\mathrm{ee}} \rho_{r,s}
   \end{align} $$

In [18]:
def calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix):
    '''Returns the Fock matrix defined by the input Hamiltonian, interaction, & density matrices.'''
    ndof = np.size(hamiltonian_matrix,0)
    fock_matrix = hamiltonian_matrix.copy()
    # Hartree potential term
    for p in range(ndof):
        for r in range(ndof):
            for orb_q in orbital_types:
                for orb_s in orbital_types:
                    for orb_t in orbital_types:
                        for orb_u in orbital_types:
                            # p, q, & t are on the same atom
                            q = ao_index(atom(p),orb_q)
                            t = ao_index(atom(p),orb_t)
                            # r, s, & u are on the same atom
                            s = ao_index(atom(r),orb_s)
                            u = ao_index(atom(r),orb_u)
                            chi_pqt = chi_on_atom(orb(p), orb_q, orb_t)
                            chi_rsu = chi_on_atom(orb(r), orb_s, orb_u)
                            fock_matrix[p,q] += 2.0*chi_pqt*chi_rsu*interaction_matrix[t,u]*density_matrix[r,s]
    # Fock exchange term
    for p in range(ndof):
        for q in range(ndof):
            for orb_r in orbital_types:
                for orb_s in orbital_types:
                    for orb_t in orbital_types:
                        for orb_u in orbital_types:
                            # r, q, & t are on the same atom
                            r = ao_index(atom(q),orb_r)
                            t = ao_index(atom(q),orb_t)
                            # p, s, & u are on the same atom
                            s = ao_index(atom(p),orb_s)
                            u = ao_index(atom(p),orb_u)
                            chi_rqt = chi_on_atom(orb(r), orb_q, orb_t)
                            chi_psu = chi_on_atom(orb(p), orb_s, orb_u)
                            fock_matrix[p,q] -= chi_rqt*chi_psu*interaction_matrix[t,u]*density_matrix[r,s]
    return fock_matrix

fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix)
print(fock_matrix)

[[ 0.3  0.   0.   0. ]
 [ 0.  -0.3  0.   0. ]
 [ 0.   0.  -0.3  0. ]
 [-0.1 -0.1 -0.1 -0.4]]


which is just a sum of two tensor contractions between the $V$ tensor and the density matrix. Because the $V$ tensor is sparse, we can save a lot of computational effort by avoiding terms that we know to be zero. However, this requires more effort than simply calling "einsum", which is a very useful NumPy function that will simplify the last steps of this project. Also, there is a lot of artificial overhead with many nested for loops in Python, so this
implementation is not very efficient.

The Fock matrix defines the matrix of molecular orbitals $\phi_{i,j}$ as its eigenvectors,

$$ \sum_{q} f_{p,q} \phi_{q,r} = E_r \phi_{p,r}, $$

and the density matrix is defined by the occupied orbitals (we occupy the lowest-energy orbitals with all of the electrons available),

$$ \rho_{p,q} = \sum_{i \in \mathrm{occ}} \phi_{p,i} \phi_{q,i}. $$

In [19]:
def calculate_density_matrix(fock_matrix):
    '''Returns the 1-electron density matrix defined by the input Fock matrix.'''
    num_occ = 3*np.size(fock_matrix,0)//orbitals_per_atom # there are 3 occupied electrons per atom
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
    occupied_matrix = orbital_matrix[:,:num_occ]
    density_matrix = occupied_matrix @ occupied_matrix.T
    return density_matrix

density_matrix = calculate_density_matrix(fock_matrix)
print(density_matrix)

[[ 0.  -0.  -0.   0.2]
 [-0.   1.  -0.   0. ]
 [-0.  -0.   1.   0. ]
 [ 0.2  0.   0.   1. ]]


To calculate a $\rho_{p,q}$ and $f_{p,q}$ that are consistent with each other, we mix in a fraction of the new density matrix with the old density matrix and re-calculate them until convergence. This is a crude strategy for an SCF cycle, but it works for weakly interacting atoms like Argon:

In [20]:
def scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix):
    '''Returns a converged density matrix defined by the input Hamiltonian, interaction, & density matrices.'''
    MAX_SCF_ITERATIONS = 100
    MIXING_FRACTION = 0.25
    CONVERGENCE_TOLERANCE = 1e-4
    for iteration in range(MAX_SCF_ITERATIONS):
        fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix)
        new_density_matrix = calculate_density_matrix(fock_matrix)

        error_norm = np.linalg.norm( density_matrix - new_density_matrix )
        if error_norm < CONVERGENCE_TOLERANCE:
            return density_matrix

        density_matrix = (MIXING_FRACTION*new_density_matrix
                          + (1.0 - MIXING_FRACTION)*density_matrix)
    print("SCF cycle didn't converge")
    return density_matrix

density_matrix = scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix)
print("density matrix after SCF:\n",density_matrix)

density matrix after SCF:
 [[ 1.4e-02 -1.4e-02 -1.4e-02  1.1e-01]
 [-1.4e-02  1.0e+00 -2.1e-04  1.7e-03]
 [-1.4e-02 -2.1e-04  1.0e+00  1.7e-03]
 [ 1.1e-01  1.7e-03  1.7e-03  9.9e-01]]


Once we have a converged density matrix, the Hartree-Fock total energy is defined as

$$ \begin{align}
  E_{\mathrm{HF}} &= E_{\mathrm{ion}} + 2 \sum_{p,q} h_{p,q} \rho_{p,q}
  + \sum_{p,q,r,s} V_{p,q,r,s} ( 2 \rho_{p,q} \rho_{r,s} - \rho_{p,s} \rho_{r,q} ) \\
  & = E_{\mathrm{ion}} + \sum_{p,q} (h_{p,q} + f_{p,q}) \rho_{p,q}
  \end{align} $$

In [21]:
def calculate_energy_hf(hamiltonian_matrix, fock_matrix, density_matrix):
    '''Returns the Hartree-Fock total energy defined by the input Hamiltonian, Fock, & density matrices.'''
    energy_hf = energy_ion + np.einsum('pq,pq',hamiltonian_matrix + fock_matrix,density_matrix)
    return energy_hf

fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix)
energy_hf = calculate_energy_hf(hamiltonian_matrix, fock_matrix, density_matrix)
print("Hartree-Fock energy =",energy_hf)

Hartree-Fock energy = -4.151238488585861


## 4. 2nd-order Moller-Plesset (MP2) perturbation theory

The Hartree-Fock approximation does not describe the London dispersion interaction between Argon atoms. We must use many-body perturbation theory to improve our approximation to the ground state of $\hat{H}$. Thankfully, we only need to use the first correction beyond Hartree-Fock theory, which is [second-order Moller-Plesset (MP2) perturbation theory](https://en.wikipedia.org/wiki/Møller–Plesset_perturbation_theory). It is simple, but computationally expensive. The computational bottleneck is the transformation of the tensor $V_{p,q,r,s}$ from the atomic orbital basis to the molecular orbital basis,

$$ \tilde{V}_{a,i,b,j} = \sum_{p,q,r,s} \phi_{p,a} \phi_{q,i} \phi_{r,b} \phi_{s,j} V_{p,q,r,s}. $$

where we are restricting some indices to occupied orbitals ($i$ and $j$) and some indices to the unoccupied "virtual" orbitals ($a$ and $b$). This use of specific orbital labels to denote a restriction of indices is standard notation in quantum chemistry. To make this easier to program, we can partition the molecular orbitals and their energies into occupied and virtual:

In [22]:
def partition_orbitals(fock_matrix):
    '''Returns a list with the occupied/virtual energies & orbitals defined by the input Fock matrix.'''
    num_occ = 3*np.size(fock_matrix,0)//orbitals_per_atom # there are 3 occupied electrons per atom
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
    occupied_energy = orbital_energy[:num_occ]
    virtual_energy = orbital_energy[num_occ:]
    occupied_matrix = orbital_matrix[:,:num_occ]
    virtual_matrix = orbital_matrix[:,num_occ:]

    return occupied_energy, virtual_energy, occupied_matrix, virtual_matrix

occupied_energy, virtual_energy, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)

print("occupied orbital energies:\n", occupied_energy)
print("virtual orbital energies:\n", virtual_energy)

occupied orbital energies:
 [-0.6 -0.3 -0.2]
virtual orbital energies:
 [0.3]


We use the factored form of $V_{p,q,r,s}$ to reduce our intermediate memory usage,

$$ \begin{align}
   \tilde{\chi}_{a,i,p} &= \sum_{q,r} \phi_{q,a} \phi_{r,i} \chi_{q,r,p} \\
   \tilde{V}_{a,i,b,j} &= \sum_{p,q} \tilde{\chi}_{a,i,p} V_{p,q}^{\mathrm{ee}} \tilde{\chi}_{b,j,q}
   \end{align} $$

In [23]:
def transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix):
    '''Returns a transformed V tensor defined by the input occupied, virtual, & interaction matrices.'''
    ndof = np.size(interaction_matrix,0)
    chi_tensor = np.zeros( (ndof,ndof,ndof) )
    for p in range(ndof):
        for q in range(ndof):
            for r in range(ndof):
                if atom(p) == atom(q) and atom(q) == atom(r):
                    chi_tensor[p,q,r] = chi_on_atom(orb(p), orb(q), orb(r))

    chi2_tensor = np.einsum('qa,ri,qrp', virtual_matrix, occupied_matrix, chi_tensor, optimize=True)
    interaction_tensor = np.einsum('aip,pq,bjq->aibj', chi2_tensor, interaction_matrix, chi2_tensor, optimize=True)
    return interaction_tensor

interaction_tensor = transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix)
print(interaction_tensor)

[[[[9.4e-02 1.0e-17 1.9e-03]]

  [[6.9e-18 9.6e-02 6.9e-18]]

  [[1.9e-03 6.9e-18 9.4e-02]]]]


In this simple implementation, we are not taking full advantage of the sparisty of the initial $\chi$ tensor. Unlike the $V$ tensor, the $\tilde{V}$ tensor is constructed explicitly and stored in memory. The MP2 correction to the total energy is then just a large summation over previously computed and stored quantities,

$$ E_{\mathrm{MP2}} = - \sum_{a,b \in \mathrm{virt}} \sum_{i,j \in \mathrm{occ}} \frac{2 \tilde{V}_{a,i,b,j}^2 - \tilde{V}_{a,i,b,j} \tilde{V}_{a,j,b,i}}{E_a + E_b - E_i - E_j} . $$

In [24]:
def calculate_energy_mp2(fock_matrix, interaction_matrix):
    '''Returns the MP2 contribution to the total energy defined by the input Fock & interaction matrices.'''
    E_occ, E_virt, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)
    V_tilde = transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix)

    energy_mp2 = 0.0
    num_occ = len(E_occ)
    num_virt = len(E_virt)
    for a in range(num_virt):
        for b in range(num_virt):
            for i in range(num_occ):
                for j in range(num_occ):
                    energy_mp2 -= ( (2.0*V_tilde[a,i,b,j]**2 - V_tilde[a,i,b,j]*V_tilde[a,j,b,i])
                                    /(E_virt[a] + E_virt[b] - E_occ[i] - E_occ[j]) )
    return energy_mp2

energy_mp2 = calculate_energy_mp2(fock_matrix, interaction_matrix)
print(energy_mp2)

-0.0224906541495073


The bottleneck of MP2 calculations is the formation of $\tilde{V}$ from $\tilde{\chi}$ and $V^{\mathrm{ee}}$, which scales as $O(n^5)$ operations for $n$ atoms because of 4 free tensor indices and 1 summed tensor index once the contraction has been optimally arranged into intermediate operations.

## (Extra Credit) 5. Semiempirical model parameterization

For a semiempirical model to be useful, it must be parameterized to fit a set of reference data, usually from experiments or high-accuracy quantum chemistry simulation data. The parameters in this project have already been fit to data, which was carried out using the fitting procedure implemented in this section. Data fitting is an optimization problem, and we can make use of pre-existing Python optimization tools by encapsulating the fitting process into a function that inputs a list of parameters and outputs a fitting error that we want to minimize.

* data to be fit
* encapsulate a function to generate input/output data pairs
* parameter -> error function
* call Python optimization

In [25]:
def calculate_total_energy(atomic_coordinates):
    '''Returns the HF & MP2 contributions to total energy'''
ndof = len(atomic_coordinates)*orbitals_per_atom

    hamiltonian_matrix = calculate_hamiltonian_matrix(atomic_coordinates)
    interaction_matrix = calculate_interaction_matrix(atomic_coordinates)
    density_matrix = calculate_atomic_density_matrix(atomic_coordinates)

    density_matrix = scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix)
    fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix)

IndentationError: unexpected indent (<ipython-input-25-bc28e76ecf8f>, line 5)

Visualize fitting process w/ inline matplotlib graphs?

## (Extra Credit) 6. Localized perturbation theory

Can I relate this section to SAPT?

An important area of ongoing research is the use of localization to accelerate QM calculations. Noble gases are an extreme example that enable localization to be applied very aggressively. If we will treat $t_{p,q}^{\mathrm{SK}}$ as a perturbation, then the Hartree-Fock equations are solved by our initial atomic guess for $\rho_{p,q}$. The energy contributions from pairs of atoms decays rapidly with distance, and we can ignore pairs of well-separated atoms.

In [ ]:
max_distance = 15.0
nearby_pairs = [ (i,j) for i in range(num_atoms)
                       for j in range(num_atoms)
                       if i < j
                       and np.linalg.norm(coordinates[i]-coordinates[j]) <= max_distance ]
print(nearby_pairs)

This is a simple but inefficient method of forming the list of nearby atomic pairs.

The localized Hartree-Fock energy can be computed from the independent-atom solutions and a localized sum of pairwise atomic contributions that combine ion-ion, electron-ion, and electron-electron Coulomb energy contributions.

In [ ]:
v_ei_onsite = eval_kernel(v_ei_kernel)([0,0,0],[0,0,0])
v_ee_onsite = eval_kernel(v_ee_kernel)([0,0,0],[0,0,0])
energy_hf_local = num_atoms*Z*(E_P + v_ei_onsite + 0.5*Z*v_ee_onsite)

v_atom = eval_kernel(v_ii_kernel + Z*v_ei_kernel + (Z**2)*v_ee_kernel)
for pair in nearby_pairs:
    r1 = coordinates[pair[0]]
    r2 = coordinates[pair[1]]
    energy_hf_local += v_atom(r1,r2)

In addition to a localized version of the MP2 energy, we must also account for the perturbative correction to the Hartree-Fock energy upon introducing $t_{p,q}^{\mathrm{SK}}$,

$$ E_{\Delta HF} = - 2 \sum_{a \in \mathrm{virt}} \sum_{i \in \mathrm{occ}} \frac{(t_{a,i}^{\mathrm{SK}})^2}{E_a - E_i} . $$

The localized MP2 correction is identical to the standard correction, but the $\tilde{V}$ tensor can be replaced by the $V$ tensor and we can avoid storing and transforming an explicit tensor. In the atomic solution, the $p$ orbitals are occupied and the $s$ orbitals are virtual.

In [ ]:
energy_hf_delta = 0.0
for pair in nearby_pairs:
    r1 = coordinates[pair[0]]
    r2 = coordinates[pair[1]]
    for o in p_set:
        energy_hf_delta -= 2.0*(t_sk('s',o,r1-r2)**2 + t_sk(o,'s',r1-r2)**2) / (E_S - E_P)

energy_mp2_local = 0.0
for o1 in p_set:
    for o2 in p_set:
        a = index(0,'s')
        b = index(0,'s')
        i = index(0,o1)
        j = index(0,o2)
        energy_mp2_local -= num_atoms*((2.0*v_tensor(a,i,b,j)**2
                                        - v_tensor(a,i,b,j)*v_tensor(a,j,b,i))
                                        / (2.0*E_S - 2.0*E_P))
for pair in nearby_pairs:
    for o1 in p_set:
        for o2 in p_set:
            a = index(pair[0],'s')
            b = index(pair[1],'s')
            i = index(pair[0],o1)
            j = index(pair[1],o2)
            energy_mp2_local -= 4.0*v_tensor(a,i,b,j)**2 / (2.0*E_S - 2.0*E_P)

print(energy_hf_local,energy_hf_delta,energy_mp2_local)

Except for the formation of the nearby atomic pairs list, this entire localized calculation scales as $O(n)$ operations for $n$ atoms. This is an example of a QM calculation that has been stripped down to the point where it can be competitive in cost with simple interatomic potentials.